**Name** : Satyaprakash Gupta
**Email id** : satyaprakashgupta9975@gmail.com

In [ ]:
!pip install q kaggle

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from google.colab import autoviz
import seaborn as sns

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satyaprakashgupta","key":"7a9ddf36a4d3697d090512525b428e40"}'}

In [ ]:
!mkdir ~/.kaggle1

In [ ]:
!cp kaggle.json ~/.kaggle/

In [29]:
! chmod 600 ~/.kaggle/kaggle.json

In [30]:
! kaggle datasets list

ref                                                         title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/customer-shopping-trends-dataset          Customer Shopping Trends Dataset                 146KB  2023-10-05 06:45:37           8020        188  1.0              
nelgiriyewithana/top-spotify-songs-2023                     Most Streamed Spotify Songs 2023                  47KB  2023-08-26 11:04:57          36365       1077  1.0              
bkcoban/customer-transactions                               Customer Transactions                              1MB  2023-10-15 17:59:17            826         22  1.0              
yakhyojon/tiktok                                            TikTok User Engagement Data        

In [31]:
!kaggle datasets download -d hhs/health-insurance-marketplace

 99% 817M/829M [00:11<00:00, 105MB/s]
100% 829M/829M [00:11<00:00, 76.9MB/s]


In [32]:
!unzip health-insurance-marketplace.zip -d data/

Archive:  health-insurance-marketplace.zip
  inflating: data/BenefitsCostSharing.csv  
  inflating: data/BusinessRules.csv  
  inflating: data/Crosswalk2015.csv  
  inflating: data/Crosswalk2016.csv  
  inflating: data/Network.csv        
  inflating: data/PlanAttributes.csv  
  inflating: data/Rate.csv           
  inflating: data/ServiceArea.csv    
  inflating: data/database.sqlite    
  inflating: data/hashes.txt         
  inflating: data/raw/._.sentinel    
  inflating: data/raw/._2014         
  inflating: data/raw/._2015         
  inflating: data/raw/._2016         
  inflating: data/raw/._Benefits_Cost_Sharing_PUF.csv  
  inflating: data/raw/._Benefits_Cost_Sharing_PUF_2015-12-08.csv  
  inflating: data/raw/._Business_Rules_PUF.csv  
  inflating: data/raw/._Business_Rules_PUF_2015-12-08.csv  
  inflating: data/raw/._Business_Rules_PUF_Reformat.csv  
  inflating: data/raw/._Machine_Readable_PUF_2015-12-21.xlsx  
  inflating: data/raw/._Network_PUF.csv  
  inflating: data/raw/.

In [ ]:
display(dbutils.fs.ls('file:/databricks/driver/data/'))

In [ ]:
source = 'file:/databricks/driver/data/'
target = '/mnt/data/'

dbutils.fs.cp(source, target, recurse=True)

In [ ]:
dbutils.fs.refreshMounts()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, mean, round, expr
spark = SparkSession.builder.appName("health").getOrCreate()

In [ ]:
data = spark.read.csv("/mnt/data/Rate.csv", header=True, inferSchema=True)
data = data.select("Year", "InsurancePlanID", "PolicyAge", "PolicyRate")

data_filtered = data.filter(col('PolicyAge') != 'Family Option')

condition_child = (data_filtered['PolicyAge'] == '0-20')
condition_adult = (data_filtered['PolicyAge'] != '0-20')

data_filtered = data_filtered.withColumn('AgeGroup', when(condition_child, 'Young').when(condition_adult, 'Older').otherwise(None))

result = data_filtered.groupBy('Year', 'InsurancePlanID', 'AgeGroup') \
                 .agg(round(mean('PolicyRate'), 2).alias('AverageRate')) \
                 .fillna({'AgeGroup': 'All'}).fillna({'AverageRate': 0})

result.show()

In [ ]:
import re
from pyspark.sql.types import StringType

In [ ]:
def categorize_age_group(text_data):
    child_match_found = re.search(r'Child', text_data, re.IGNORECASE)
    adult_match_found = re.search(r'Adult', text_data, re.IGNORECASE)

    if child_match_found and not adult_match_found:
        return 'AgeGroup: Child'
    elif adult_match_found and not child_match_found:
        return 'AgeGroup: Adult'
    else:
        return 'AgeGroup: Unknown'

insurance_records = spark.read.csv('/mnt/data/BenefitsCostSharing.csv', header=True, inferSchema=True)
insurance_records = insurance_records.select("CoverageBenefitName", "PolicyYear", "PolicyIdentifier", "LocationCode")

insurance_records = insurance_records.withColumn("PolicyIdentifier", expr("substring(PolicyIdentifier, 1, length(PolicyIdentifier)-3)"))

categorize_age_group_udf = udf(categorize_age_group, StringType())
insurance_records = insurance_records.withColumn("AgeGroupCategory", categorize_age_group_udf(insurance_records["CoverageBenefitName"]))
insurance_records = insurance_records.na.drop()

merged_insurance_data = insurance_records.join(rate_pivot, on=['PolicyYear', 'PolicyIdentifier', 'AgeGroupCategory'], how='left')
merged_insurance_data = merged_insurance_data.withColumnRenamed("AverageRate", "PolicyAverageRate")
merged_insurance_data = merged_insurance_data.fillna(0, subset=['PolicyAverageRate'])

merged_insurance_data.show()

In [ ]:
!mkdir transformed

In [ ]:
final_data = merged_data.repartition(1)
final_data.write.csv('file:/databricks/driver/transformed/final.csv', header=True, mode='overwrite')

In [ ]:
display(dbutils.fs.ls('file:/databricks/driver/transformed/final.csv/'))

In [ ]:
source = 'file:/databricks/driver/transformed/'
target = '/mnt/transformed/'

dbutils.fs.cp(source, target, recurse=True)

In [ ]:
dbutils.fs.refreshMounts()

In [ ]:
display(dbutils.fs.ls('/mnt/transformed/final.csv/'))

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("health").getOrCreate()

directory_path = '/mnt/transformed/final.csv/'

files = dbutils.fs.ls(directory_path)

csv_files = [file for file in files if file.name.endswith('.csv')]

file_path = directory_path + csv_files[0].name

print(file_path)